In [1]:
import numpy as np
reward = np.array([[0, 0.2],
                  [0, 0.2],
                  [0, 0.2],
                  [0, 0.2],
                  [1, 0.2]])

transition_prob = np.array([
    [
        [0, 0.8, 0.2, 0, 0],
        [0, 0, 0.8, 0.2, 0],
        [0, 0, 0.2, 0.8, 0],
        [0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1]
    ],
    [
        [0.9, 0.1, 0, 0, 0],
        [0.9, 0.1, 0, 0, 0],
        [0.9, 0, 0.1, 0, 0],
        [0.9, 0, 0.1, 0, 0],
        [0.9, 0, 0.1, 0, 0]
    ]
])

initial_state = np.array([[1, 0, 0, 0, 0]])

gamma = 0.95 #discount factor

## DP

To compute the optimal Q-values using Value Iteration, we have to do:

1. Initialize the Q-values $Q(s, a)$ with zeros or some other initial values.
2. Update the Q-values using the Bellman optimality equation for Q-values:

$$
Q^{*}(s, a) = \sum_{s'} P(s'|s, a) \left[ R(s, a) + \gamma \max_{a'} Q^{*}(s', a') \right]
$$

3. Repeat step 2 until the Q-values converge (i.e., the maximum change in Q-values between consecutive iterations is smaller than a predefined threshold).

Once the process converges, the optimal Q-values $Q^*(s, a)$ are obtained.

In [2]:
def Q_value_DP(reward, transition_prob, gamma):
    q_table = np.zeros((5,2))
    while True:
        q_table_new = np.copy(q_table)#np.zeros((5,2))
        for s in range(5):
            for a in range(2):
                q_table_new[s ,a] = np.sum(transition_prob[a, s, :] * (reward[s, a] + gamma * np.max(q_table, axis=1)))

        if np.max(np.abs(q_table - q_table_new)) < 1e-6:
            break
        q_table = q_table_new
    
    return q_table

q_value_dp = Q_value_DP(reward, transition_prob, gamma)
print("Q table DP: ")
print(q_value_dp)

Q table DP: 
[[16.42770906 15.87575943]
 [17.15862264 15.87575943]
 [17.82714116 15.93926869]
 [18.99998066 15.93926869]
 [19.99998066 15.93926869]]


## Q-Learning


The Q-learning update rule is given as:

$$
Q(s, a) \leftarrow Q(s, a) + \alpha \left[ r + \gamma \max_{a'} Q(s', a') - Q(s, a) \right]
$$

For the Q-learning algorithm we are following the following steps:

1. Initialize the Q-values $Q(s, a)$ with zeros.
2. For each episode, start from the initial state ($s_0$) and follow these steps:
   a. Choose an action $a$ from the current state $s$ using epsilon-greedy.
   b. Take the action $a$, observe the reward $r$ and the next state $s'$.
   c. Update the Q-value using the Q-learning update rule.
   d. Set the current state $s$ to the next state $s'$.
   e. Repeat steps a-d until the max_episode times (because we dont have terminal state).

3. Continue running episodes until reach maximum number of episodes.

After the Q-learning algorithm converges, the optimal Q-values $Q^*(s, a)$ are obtained.

In [3]:
def epsilon_greedy(q_values, state, epsilon):
    # print(len(q_values[state]))
    if np.random.rand() < epsilon:
        return np.random.choice(len(q_values[state]))
    else:
        return np.argmax(q_values[state])

def Q_learning(reward, gamma, alpha, initial_state = 0, max_episode = 1000, max_step = 1000):
    q_table = np.zeros((5,2))
    for episod in range(max_episode):
        state = initial_state
        for step in range(max_step):
            action = epsilon_greedy(q_table, state, 0.1)
            state_new = np.random.choice(5, p=transition_prob[action, state])                    
            q_table[state, action] = q_table[state, action] + alpha *(reward[state, action] + gamma * np.max(q_table[state_new]) - q_table[state, action]) 
            state = state_new

    return q_table

q_learning = Q_learning(reward, gamma, 0.1)
print("Q table Q-Learning: ")
print(q_learning)

Q table Q-Learning: 
[[16.45047984 15.73941514]
 [17.25684472 15.78435154]
 [17.67300918 15.8645954 ]
 [19.         15.85974815]
 [20.         15.93704007]]


## Deference between DP and Q-Learning resluts
The difference between DP and Q-Learning results is quite small, as we can see. The reason for this difference is that for Q-Learning, we ran the algorithm for only 1000 episodes with 1000 steps each. If we were to increase the number of episodes and steps, the difference between the two algorithms should become smaller and smaller.